In [1]:
import json
import os
import pandas as pd

In [2]:
os.chdir('../')

In [3]:
from src.data.ClickhouseClient import ClickhouseClient

In [4]:
# Get Clickhouse credentials and connect
filename_credentials = '../.credentials/clickhouse_credentials.json'
with open(filename_credentials, 'r') as f:
    credentials = json.load(f)

In [7]:
client = ClickhouseClient(
    host = credentials['host'],
    port = credentials['port'],
    user = credentials['user'],
    password = credentials['password'],
    database = credentials['database']
)

ClickhouseClient.py: connection to 35.210.170.130 done


In [8]:
# Drop raw schema if not exists, and then it's created again
database = 'raw'
client.drop_database(database)
client.create_database(database)

ClickhouseClient.py: database raw created
ClickhouseClient.py: database raw created


In [10]:
# Read initialize config file
filename_init_config = './clickhouse_config.json'
with open(filename_init_config, 'r') as f:
    config = json.load(f)

In [13]:
# Read and create the following tables in raw
for t in config['files']:
    # Build create sentence
    sentence_first = "CREATE TABLE"

    ## Check exists flag
    sentence_if = " IF NOT EXISTS "

    ## Get fields and format the sentence
    list_fields = list()
    for s in t['schema']:
        list_fields.append(s['name'] + ' ' + s['type'])

    ## Whole sentence
    sentence = sentence_first + sentence_if + database + '.' + t['table_name'] + ' (' + ', '.join(list_fields) + ') ENGINE = Memory'

    print(sentence)

CREATE TABLE IF NOT EXISTS raw.meal (meal_id Int64, category String, cuisine String) ENGINE = Memory
CREATE TABLE IF NOT EXISTS raw.center (center_id Int64, city_code Int32, region_code Int32 , center_type String, op_area Float32) ENGINE = Memory
CREATE TABLE IF NOT EXISTS raw.train (id Int64, week Int16, center_id Int64, meal_id Int64, checkout_price Float32, base_price Float32, emailer_for_promotion Int8, homepage_featured Int8, num_orders Int32) ENGINE = Memory
CREATE TABLE IF NOT EXISTS raw.test (id Int64, week Int16, center_id Int64, meal_id Int64, checkout_price Float32, base_price Float32, emailer_for_promotion Int8, homepage_featured Int8) ENGINE = Memory
